# 인공 신경망 구조 이해하기

In [10]:
!pip install torchvision

In [20]:
def imshow(img):
    # img = img / 2 + 0.5     # denormalize
    npimg = img.numpy()
    plt.axis('off')
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()

dataiter = iter(train_data) # iterator
images, labels = dataiter.next()

imshow(torchvision.utils.make_grid(images))

AttributeError: 'iterator' object has no attribute 'next'

In [34]:
len(train_data[0][0][0][0])

28

In [134]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
import torchvision

transform=torchvision.transforms.ToTensor()
train_data = torchvision.datasets.MNIST('./data', train=True, download=True, transform=transform)
test_data = torchvision.datasets.MNIST('./data', train=False, download=True, transform=transform)

print(train_data)

# Write neural network code
class NeuralNetwork(nn.Module):

    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.input_layer = nn.Linear(784, 1568)
        self.activation = nn.ReLU()
        self.hidden_layers = [nn.Linear(1568, 1568) for _ in range(1)]
        self.output_layer = nn.Linear(1568, 10)
        
    def forward(self, x):
        x = self.input_layer(x)
        x = self.activation(x)
        for i in self.hidden_layers:
            x=i(x)
            self.activation(x)
        x = self.output_layer(x)
        return x

# Define the network and the loss function.
network = NeuralNetwork()
# loss_function = nn.MSELoss()
loss_function = nn.CrossEntropyLoss()

# Steps for gradient descent.
# 'lr' stands for learning rate.
optimizer = torch.optim.Adam(network.parameters(), lr=0.001)

# Training one step with sample data
x = train_data[2][0].view(1, 784)
y = [0 for i in range(10)]
y[train_data[2][1]]=1
y = torch.Tensor(y)
y= y.view(1,10)


optimizer.zero_grad()
output = network(x)
# print(output.shape, y.shape)
loss = loss_function(y, output)
loss.backward()
optimizer.step()

# Check parameters
parameters = network.input_layer.state_dict()
w = parameters['weight']  # e.g. [-5, -1.2, 1.2, 1.2, 2, 5]
b = parameters['bias']  # e.g. [-7.7, -1.3, 1, -0.2, -1.1, -5]
print(f"loss: {loss}")
print(w)
print(b)

Dataset MNIST
    Number of datapoints: 60000
    Root location: ./data
    Split: Train
    StandardTransform
Transform: ToTensor()
loss: -0.27907595038414
tensor([[ 0.0177,  0.0106,  0.0129,  ...,  0.0066, -0.0196, -0.0165],
        [-0.0052,  0.0302, -0.0050,  ...,  0.0348, -0.0267, -0.0127],
        [ 0.0152,  0.0307, -0.0270,  ...,  0.0119,  0.0083, -0.0011],
        ...,
        [-0.0223,  0.0189, -0.0187,  ..., -0.0190,  0.0095, -0.0134],
        [-0.0190, -0.0013,  0.0349,  ..., -0.0036,  0.0265, -0.0180],
        [-0.0064, -0.0289,  0.0264,  ...,  0.0250, -0.0117, -0.0133]])
tensor([ 0.0311, -0.0061,  0.0264,  ...,  0.0186,  0.0329, -0.0149])


In [135]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import time
import random
import numpy

# Define the network and the loss function.
network = NeuralNetwork()
# loss_function = nn.MSELoss()
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(network.parameters(), lr=0.01)

pbar = tqdm(range(50000), desc="Loss: --")
for epoch in pbar:
    test_case=random.randint(0, 59999)
    x = train_data[test_case][0].view(1, 784)
    y = [0 for i in range(10)]
    y[train_data[test_case][1]]=1
    y = torch.Tensor(y)
    y= y.view(1,10)
    # time.sleep(1)

    optimizer.zero_grad()
    output = network(x)
    loss = loss_function(output, y)
    loss.backward()
    optimizer.step()
    
    if epoch % 100 == 0:
        pbar.set_description(f"Loss: {loss.item():.3f}")

Loss: 2.612:  69%|██████▉   | 34699/50000 [11:10<04:55, 51.72it/s]  


KeyboardInterrupt: 

In [119]:
x = test_data[2][0].view(1, 784)
y = [0 for i in range(10)]
y[test_data[2][1]]=1
y = torch.Tensor(y)
y= y.view(1,10)


optimizer.zero_grad()
output = network(x)
loss = loss_function(y, output)
loss.backward()
optimizer.step()
print(f"loss: {loss}, output: {(output[0])}, answer: {y}")

loss: -14.979963302612305, output: tensor([-6.0854, 14.5195, -5.5369, -1.3197, -0.7551, -6.0401, -0.6027,  2.7124,
         1.0213,  1.8996], grad_fn=<SelectBackward0>), answer: tensor([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [136]:
end=0
pbar = tqdm(range(5000), desc="Loss: --")
for epoch in pbar:
    x = test_data[epoch][0].view(1, 784)
    y = [0 for i in range(10)]
    y[test_data[epoch][1]]=1
    y = torch.Tensor(y)
    y= y.view(1,10)
    # time.sleep(1)

    optimizer.zero_grad()
    output = network(x)
    loss = loss_function(output, y)
    loss.backward()
    optimizer.step()
    output=output.tolist()[0]
    
    if output.index(max(output))==test_data[epoch][1]:
        end+=1

    if epoch % 100 == 0:
        pbar.set_description(f"Loss: {loss.item():.3f}")

print(end/epoch*100)

Loss: 1.974: 100%|██████████| 5000/5000 [01:37<00:00, 51.30it/s] 

52.830566113222645


In [83]:
type((output.detach().numpy()))
type(numpy.array([1,2,3,4]))

numpy.ndarray

In [87]:
output.detach().numpy().tolist

[array([0.09532588, 0.13042647, 0.08980362, 0.13609603, 0.0969222 ,
        0.12474118, 0.06336303, 0.1024522 , 0.0912748 , 0.07248389],
       dtype=float32)]

In [84]:
numpy_array = output.detach().numpy()
plt.plot(numpy.array([i for i in range(10)]).view(1, 10).detach().numpy(), output.detach().numpy())

ValueError: Type must be a sub-type of ndarray type

In [ ]:
# plot the result
x_lim = 5
num_points = 1

x = torch.linspace(-x_lim, x_lim, num_points)
y = x**3 + x**2 - x - 1
# y= x**4-14*x**3-83*x**2+168*x+540
plt.plot(x, y, label="ground truth", color="black")
plt.plot(x, network(x[:, None]).detach().numpy(), label="prediction", color="red")
plt.legend()
plt.show()